In [ ]:
]activate ../..

In [ ]:
using Dates
# using Profile

In [ ]:
include("real.jl")
using .RealIE: get_params, get_initial_trace, get_score

In [ ]:
using .RealIE: run_tracked_splitmerge_inference!, write_state!, get_state

In [ ]:
sparsity = 20/1999^2
beta_prior=(4, 4*(1-sparsity)/sparsity)
dirichlet_prior_val=3e-3
(params, sentences) = get_params(filename="Umass-sub-corpus/pluieTriples_2013_01_06_5.json"; beta_prior, dirichlet_prior_val, mean_n_rels=130, num_relations_var=200)

In [ ]:
initial_tr = get_initial_trace(sentences, params)
nothing

In [ ]:
function save_state!(tr, i, dirname)
    f = open(joinpath(dirname, "$i"), "w")
    write_state!(f, get_state(tr))
    close(f)
end
function run_inference_with_saving!(run_inference!, initial_tr, num_iters, examine!, other_args...; save_freq=1, examine_freq=1, kwargs...)
    datetime = Dates.format(now(), "yyyymmdd-HH_MM_SS")
    dirname = joinpath(realpath(joinpath(@__DIR__, "../../out/runs")), datetime)
    mkdir(dirname)
    println("Will save to $dirname")
    println()

    function saving_examine!(i, tr)
        if i % save_freq === 0
            save_state!(tr, i, dirname)
        end
        examine!(i, tr)
    end

    run_inference!(initial_tr, num_iters, saving_examine!, other_args...; examine_freq=gcd(save_freq, examine_freq), kwargs...)
end

In [ ]:
examine!(i, tr) = println("Score at iteration $i : $(get_score(tr)) | num rels = $(tr[:world => :num_relations => ()])")

In [ ]:
success = false
for _=1:10
  success && break;
  try
    run_inference_with_saving!(run_tracked_splitmerge_inference!, initial_tr, 1500000, examine!;
        save_freq=120, examine_freq=30, log=true, log_freq=120
    )
    success = true
  catch e
#     Profile.print()
    println("caught an error!:")
    display(e)
    e isa InterruptException && break;
  end
end